# 1. Datascraping de rembrandt schilderijen

Hiervoor moeten we eerst de nodige imports doen 

In [1]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

De url waar we de schilderijen van halen steken we in een variabele, ook de folder waar we ze willen opslaan

In [2]:
BASE_URL = "http://www.rembrandtpainting.net/"
# Gebruik absoluut pad vanaf de workspace root
SAVE_FOLDER = os.path.join(os.path.dirname(os.getcwd()), "datasets", "raw", "Rembrandt")

Controleer dat de map wel degelijk bestaat

In [3]:
if not os.path.exists(SAVE_FOLDER):
    os.makedirs(SAVE_FOLDER)
    print(f"Map aangemaakt: {SAVE_FOLDER}")

Map aangemaakt: /datasets/raw/Rembrandt


Nu moeten we een functie schrijven die HTML vanuit de url kan halen en we gebruiken hiervoor Beautiful soup dat een package is die vaak gebruikt wordt voor webscrapers

In [4]:
def get_soup(url):
    """Haalt de HTML op en maakt er een BeautifulSoup object van."""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return BeautifulSoup(response.text, 'html.parser')
    except Exception as e:
        print(f"Fout bij ophalen {url}: {e}")
        return None


Hierna schrijven een functie die de schilderijen uit deze html kan halen en ze downloaden

In [5]:
def download_image(img_url, filename):
    """Downloadt een enkele afbeelding."""
    try:
        img_data = requests.get(img_url, timeout=15).content
        path = os.path.join(SAVE_FOLDER, filename)
        
        # Alleen opslaan als het bestand nog niet bestaat of leeg is
        if not os.path.exists(path) or os.path.getsize(path) == 0:
            with open(path, 'wb') as f:
                f.write(img_data)
            print(f"Gedownload: {filename}")
        else:
            print(f"Bestaat al: {filename}")
            
    except Exception as e:
        print(f"Fout bij downloaden {filename}: {e}")


Dit is de functie die alles zal uitvoeren

In [6]:
def scrape_rembrandt():
    print("Start scraping van rembrandtpainting.net...")
    
    # 1. Haal de homepagina op om categorieën te vinden
    soup = get_soup(BASE_URL)
    if not soup:
        return

    # Trefwoorden om categorie-links te vinden (op basis van site structuur)
    category_keywords = ["History Painting", "Landscape", "Self Portrait", "Portraits", "Genre", "Etchings"]
    category_links = set()

    # Zoek alle links op de voorpagina
    for a_tag in soup.find_all('a', href=True):
        link_text = a_tag.get_text().strip()
        href = a_tag['href']
        
        # Als een van de trefwoorden in de linktekst staat, is het waarschijnlijk een galerij
        if any(keyword.lower() in link_text.lower() for keyword in category_keywords):
            full_url = urljoin(BASE_URL, href)
            category_links.add(full_url)

    print(f"Gevonden categorie-pagina's: {len(category_links)}")

    # 2. Ga elke categorie pagina af
    total_downloaded = 0
    
    for cat_url in category_links:
        print(f"\nBezig met categorie: {cat_url}")
        cat_soup = get_soup(cat_url)
        if not cat_soup:
            continue
            
        # Zoek naar afbeeldingen op de categorie pagina
        # Vaak zijn afbeeldingen in 'img' tags, soms gewrapped in een 'a' tag naar de high-res versie.
        
        images_found = cat_soup.find_all('img')
        
        for img in images_found:
            src = img.get('src')
            if not src:
                continue
                
            # Filter rommel (kleine icoontjes, navigatiepijltjes, logos)
            if any(x in src.lower() for x in ['bullet', 'button', 'logo', 'icon', 'arrow', 'spacer']):
                continue

            # Check of de afbeelding in een link zit (vaak linkt de thumbnail naar de grote versie)
            parent = img.find_parent('a')
            if parent and parent.get('href'):
                possible_hires_link = parent.get('href')
                if possible_hires_link.lower().endswith(('.jpg', '.jpeg', '.png')):
                    image_url = urljoin(cat_url, possible_hires_link)
                else:
                    # Geen directe link naar jpg, gebruik de img src
                    image_url = urljoin(cat_url, src)
            else:
                image_url = urljoin(cat_url, src)

            # Naamgeving bestand
            filename = os.path.basename(urlparse(image_url).path)
            
            # Sla op als "rembrandt_..." om conflicten te vermijden
            if not filename.lower().startswith('rembrandt'):
                filename = f"rembrandt_{filename}"
                
            # Filter te korte bestandsnamen of rare extensies
            if len(filename) < 5 or not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue

            download_image(image_url, filename)
            total_downloaded += 1

    print(f"\nKlaar! Totaal aantal afbeeldingen verwerkt: {total_downloaded}")
    print(f"Check de map: {os.path.abspath(SAVE_FOLDER)}")

if __name__ == "__main__":
    scrape_rembrandt()

Start scraping van rembrandtpainting.net...
Gevonden categorie-pagina's: 11

Bezig met categorie: http://www.rembrandtpainting.net/about_rembrandt's_etchings.htm
Gevonden categorie-pagina's: 11

Bezig met categorie: http://www.rembrandtpainting.net/about_rembrandt's_etchings.htm
Gedownload: rembrandt_ad_signature.jpg
Gedownload: rembrandt_ad_signature.jpg
Gedownload: rembrandt_artcomposter_c.jpg
Gedownload: rembrandt_artcomposter_c.jpg
Bestaat al: rembrandt_ad_signature.jpg
Bestaat al: rembrandt_ad_signature.jpg
Gedownload: rembrandt_dvdrembrandtsmall.jpg
Gedownload: rembrandt_dvdrembrandtsmall.jpg
Gedownload: rembrandt_etching_print.jpg
Gedownload: rembrandt_etching_print.jpg
Gedownload: rembrandt_slf_prtrt_with_loose_hair.jpg
Gedownload: rembrandt_slf_prtrt_with_loose_hair.jpg
Gedownload: rembrandt_stove.jpg
Gedownload: rembrandt_stove.jpg
Gedownload: rembrandt_polander.jpg

Bezig met categorie: http://www.rembrandtpainting.net/rmbrndt_etchings/a_rembrandt_etchings.htm
Gedownload: re

Wat doet dit script?
1. Het scant de hoofdpagina van rembrandtpainting.net.
2. Het zoekt naar links met woorden als "History", "Landscape", "Portrait" (de categorieën van de schilderijen).
3. Het opent elke categorie-pagina en zoekt naar afbeeldingen.
4. Het is slim genoeg om te checken of een plaatje een link is naar een grotere versie (high-res), wat vaak zo is op oude sites.
5. Het filtert "rommel" zoals navigatiepijltjes en logo's weg.
6. Het slaat alles op in datasets/raw/Rembrandt.